## Analysis of EVI Trend Break Magnitudes with XGBRegressor - v5 sv3

Peter R., 2023-08-25

### Intro
There are several way to carry our Extreme Gradient Boosting (XGB). Here I use XGBRegression() to analyze forest EVI breaks (both negative & positive breaks) together.

Here I try to answer the question: What factors predict EVI (negative and positive) Trend Break magnitude? Here I run two separate models, one with negative breaks and the other with positive ones.

Note that driver data have been assigned here by using spatio-temporal matches between breaks and remote sensing derived disturbance data. So far, I only have data for three drivers: fire, harvest, & insects.  The driver data are mainly nulls as I was not able to match most of the EVI breaks with disturbance data.  An important missing driver is likely tree windthrow.

This is a simplified version of v5 meant to be run on DRAC.

#### Positive breaks XGB model

Here I run positive breaks only.

In [4]:
import os
import time

import pandas as pd
from numpy import nan
import xgboost as xgb
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


start = time.time()

# Get the current working directory
cwd = os.getcwd()

#print(cwd)

# DRAC directory
#os.chdir("/home/georod/projects/def-mfortin/georod/scripts/github/forc_trends/models/xgboost")

print("XGB version:", xgb.__version__)

# Windows
df1 = pd.read_csv(r'.\data\forest_evi_breaks_positive_sam1.csv', skipinitialspace=True)
# DRAC
#df1 = pd.read_csv(r'./data/forest_evi_breaks_positive_v1.csv', skipinitialspace=True)
#df1.head()

df2 = pd.get_dummies(df1, columns=['protected'], dtype=float)

df2= df2[df2['precipitation'].notna()]

X1 = df2.iloc[:,2:24]

X1.drop(X1.columns[[2, 12, 14, 16, 18]], axis=1,inplace=True)

y1 = df2.iloc[:,1]


features_names1 = ["age","deciduous","elevation","precipitation","temperature","precipitation_lag1", "temperature_lag1", "precipitation_lag2", "temperature_lag2", "precipitation_lag3", "temperature_lag3",
                 "rh" ,"rh_lag1","rh_lag2","rh_lag3"]


# Fine tune parameters using RandomizedSearchCV (faster)
# max_depth is tree complexity in Elith et al. 2008
# n_estimators=100 is the number of trees. Elith et al. 2008 say this should be 1000 at least
# Elith et al. 2008 suggests low learning rate

seed = 7 # random seed to help with replication
testsize1 = 0.33 # percent of records to test after training

# Split data set. Note the 'stratify' option
x1_train, x1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=testsize1, random_state=seed)


# Fine tunning parameters with Random Search
#search space
params_xgboost = {
 "learning_rate"    : [ 0.001, 0.005, 0.01, 0.05, 0.10, 0.15],
 "max_depth"        : [ 3, 4, 5, 6, 8, 10],
 "gamma"            : [ 0.0, 0.01, 0.05, 0.1, 0.2, 0.3],
 #"colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7],
 #'n_estimators'     : [5, 10, 15, 20, 25, 30, 35],
'n_estimators'     : [300],
 'objective': ['reg:squarederror'],
#'early_stopping_rounds': [10]
# reg_alpha provides L1 regularization to the weight, higher values result in more conservative models
"reg_alpha": [1e-5, 1e-2, 0.1, 1, 10, 100],
# reg_lambda provides L2 regularization to the weight, higher values result in more conservative models
"reg_lambda": [1e-5, 1e-2, 0.1, 1, 10, 100]
}

model_base1 = XGBRegressor()

random_search = RandomizedSearchCV(estimator = model_base1, 
                      param_distributions = params_xgboost, 
                      n_iter = 100, 
                      cv = 5, 
                      verbose=10, 
                      random_state=42,
                      scoring = 'neg_mean_squared_error', 
                        refit=True,
                      n_jobs = -1)

#params glare proba
random_search.fit(x1_train, y1_train)

#random_search
print(random_search.best_params_)
print(random_search.best_estimator_)

# How to early_stopping_rounds=10?
# Model with best parameters 1
model_bp1 = random_search.best_estimator_

#print(model_bp1)

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=seed)

# evaluate model with train
# -1 means using all processors in parallel
# cross val takes place withing the train data set
scores = cross_val_score(model_bp1, x1_train, y1_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

scores2 = cross_val_score(model_bp1, x1_train, y1_train, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)

# force scores to be positive
scores2 = absolute(scores)
print('Mean MSE: %.3f (%.3f)' % (scores2.mean(), scores2.std()) )

# evaluate model with variance explained
scores3 = cross_val_score(model_bp1, x1_train, y1_train, scoring='explained_variance', cv=cv, n_jobs=-1)
#print(scores3)

# force scores to be positive
#print(statistics.mean(scores3))
print('Mean Var. Explained: %.3f (%.3f)' % (scores3.mean(), scores3.std()) ) 

# R-squared
# evaluate model with variance explained
scores4 = cross_val_score(model_bp1, x1_train, y1_train, scoring='r2', cv=cv, n_jobs=-1)
#print(scores3)

# force scores to be positive
#print(statistics.mean(scores3))
print('R-sq: %.3f (%.3f)' % (scores4.mean(), scores4.std()) ) 


# EVALUATION (with test)
eval_set = [(x1_train, y1_train), (x1_test, y1_test)]
#UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
model_bp1.fit(x1_train, y1_train, eval_set=eval_set, verbose=False)
# make predictions for test data
y_pred = model_bp1.predict(x1_test)
predictions = [round(value) for value in y_pred]
# retrieve performance metrics
results = model_bp1.evals_result()

mse = mean_squared_error(y1_test, y_pred)
#r2 = explained_variance_score(y1_test, ypred)
r2 = r2_score(y1_test, y_pred)
print("MSE: %.2f" % mse)

print("RMSE: %.2f" % (mse**(1/2.0)))

print("R-sq: %.2f" % r2)

# Save model
# save in JSON format
model_bp1.save_model("model_bp1_pos_brks_sam1.json")
# save in text format
#model_m2.save_model("model_m2.txt")

end = time.time()

total_time = end-start
#total_time
print("Total time: %.2f" % total_time)

# Load model
# load saved model
#model2 = xgb.Regressor()
#model2.load_model("model_regression1.json")


# all brks
#Mean MAE: 517.899 (64.482)
#Mean MSE: 517.899 (64.482)
#Mean Var. Explained: 0.197 (0.098)
#R-sq: 0.185 (0.101)
#MSE: 497906.63
#RMSE: 705.62
#R-sq: 0.15

# Take 1, positive breaks sample 1
#Mean MAE: 255.336 (31.703)
#Mean MSE: 255.336 (31.703)
#Mean Var. Explained: 0.103 (0.129)
#R-sq: 0.086 (0.136)
#MSE: 109146.11
#RMSE: 330.37
#R-sq: 0.10
#Total time: 43.21

# Take 2
#Mean MAE: 223.763 (14.087)
#Mean MSE: 223.763 (14.087)
#Mean Var. Explained: 0.291 (0.077)
#R-sq: 0.287 (0.078)
#MSE: 102980.89
#RMSE: 320.91
#R-sq: 0.35

#Mean MAE: 255.336 (31.703)
#Mean MSE: 255.336 (31.703)
#Mean Var. Explained: 0.103 (0.129)
#R-sq: 0.086 (0.136)
#MSE: 109146.11
#RMSE: 330.37
#R-sq: 0.10
#Total time: 42.47

#Take 3, all positive breaks with alpha & lamda
#Mean MAE: 223.102 (13.814)
#Mean MSE: 223.102 (13.814)
#Mean Var. Explained: 0.303 (0.065)
#R-sq: 0.299 (0.065)
#MSE: 106925.62
#RMSE: 326.99
#R-sq: 0.32

XGB version: 1.7.6
Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'reg_lambda': 100, 'reg_alpha': 100, 'objective': 'reg:squarederror', 'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.01}
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.01, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=300, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)
XGBRegressor(base_score=None, boos

#### Negative breaks XGB model

Here I run a model with negative breaks only

In [5]:
import os
import time

import pandas as pd
from numpy import nan
import xgboost as xgb
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


start = time.time()

# Get the current working directory
cwd = os.getcwd()

#print(cwd)

# DRAC directory
#os.chdir("/home/georod/projects/def-mfortin/georod/scripts/github/forc_trends/models/xgboost")

print("XGB version:", xgb.__version__)

# Windows
df1 = pd.read_csv(r'.\data\forest_evi_breaks_negative_sam1.csv', skipinitialspace=True)
# DRAC
#df1 = pd.read_csv(r'./data/forest_evi_breaks_negative_sam1.csv', skipinitialspace=True)
#df1.head()

df2 = pd.get_dummies(df1, columns=['protected'], dtype=float)

df2= df2[df2['precipitation'].notna()]

X1 = df2.iloc[:,2:24]

X1.drop(X1.columns[[2, 12, 14, 16, 18]], axis=1,inplace=True)

y1 = df2.iloc[:,1]


features_names1 = ["age","deciduous","elevation","precipitation","temperature","precipitation_lag1", "temperature_lag1", "precipitation_lag2", "temperature_lag2", "precipitation_lag3", "temperature_lag3",
                 "rh" ,"rh_lag1","rh_lag2","rh_lag3"]


# Fine tune parameters using RandomizedSearchCV (faster)
# max_depth is tree complexity in Elith et al. 2008
# n_estimators=100 is the number of trees. Elith et al. 2008 say this should be 1000 at least
# Elith et al. 2008 suggests low learning rate

seed = 7 # random seed to help with replication
testsize1 = 0.33 # percent of records to test after training

# Split data set. Note the 'stratify' option
x1_train, x1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=testsize1, random_state=seed)


# Fine tunning parameters with Random Search
#search space
params_xgboost = {
 "learning_rate"    : [ 0.001, 0.005, 0.01, 0.05, 0.10, 0.15],
 "max_depth"        : [ 3, 4, 5, 6, 8, 10],
 "gamma"            : [ 0.0, 0.01, 0.05, 0.1, 0.2, 0.3],
 #"colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7],
 #'n_estimators'     : [5, 10, 15, 20, 25, 30, 35],
'n_estimators'     : [100],
 'objective': ['reg:squarederror'],
#'early_stopping_rounds': [10]
# reg_alpha provides L1 regularization to the weight, higher values result in more conservative models
"reg_alpha": [1e-5, 1e-2, 0.1, 1, 10, 100],
# reg_lambda provides L2 regularization to the weight, higher values result in more conservative models
"reg_lambda": [1e-5, 1e-2, 0.1, 1, 10, 100]
}

model_base1 = XGBRegressor()

random_search = RandomizedSearchCV(estimator = model_base1, 
                      param_distributions = params_xgboost, 
                      n_iter = 100, 
                      cv = 5, 
                      verbose=10, 
                      random_state=42,
                      scoring = 'neg_mean_squared_error', 
                        refit=True,
                      n_jobs = -1)

#params glare proba
random_search.fit(x1_train, y1_train)

#random_search
print(random_search.best_params_)
print(random_search.best_estimator_)

# How to early_stopping_rounds=10?
# Model with best parameters 1
model_bp1 = random_search.best_estimator_

#print(model_bp1)

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=seed)

# evaluate model with train
# -1 means using all processors in parallel
# cross val takes place withing the train data set
scores = cross_val_score(model_bp1, x1_train, y1_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

# force scores to be positive
scores = absolute(scores)
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()) )

scores2 = cross_val_score(model_bp1, x1_train, y1_train, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)

# force scores to be positive
scores2 = absolute(scores)
print('Mean MSE: %.3f (%.3f)' % (scores2.mean(), scores2.std()) )

# evaluate model with variance explained
scores3 = cross_val_score(model_bp1, x1_train, y1_train, scoring='explained_variance', cv=cv, n_jobs=-1)
#print(scores3)

# force scores to be positive
#print(statistics.mean(scores3))
print('Mean Var. Explained: %.3f (%.3f)' % (scores3.mean(), scores3.std()) ) 

# R-squared
# evaluate model with variance explained
scores4 = cross_val_score(model_bp1, x1_train, y1_train, scoring='r2', cv=cv, n_jobs=-1)
#print(scores3)

# force scores to be positive
#print(statistics.mean(scores3))
print('R-sq: %.3f (%.3f)' % (scores4.mean(), scores4.std()) ) 


# EVALUATION (with test)
eval_set = [(x1_train, y1_train), (x1_test, y1_test)]
#UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
model_bp1.fit(x1_train, y1_train, eval_set=eval_set, verbose=False)
# make predictions for test data
y_pred = model_bp1.predict(x1_test)
predictions = [round(value) for value in y_pred]
# retrieve performance metrics
results = model_bp1.evals_result()

mse = mean_squared_error(y1_test, y_pred)
#r2 = explained_variance_score(y1_test, ypred)
r2 = r2_score(y1_test, y_pred)
print("MSE: %.2f" % mse)

print("RMSE: %.2f" % (mse**(1/2.0)))

print("R-sq: %.2f" % r2)

# Save model
# save in JSON format
model_bp1.save_model("model_bp1_neg_brks_sam1.json")
# save in text format
#model_m2.save_model("model_m2.txt")

end = time.time()

total_time = end-start
#total_time
print("Total time: %.2f" % total_time)

# Load model
# load saved model
#model2 = xgb.Regressor()
#model2.load_model("model_regression1.json")

# Take 1, neg brks
#Mean MAE: 236.620 (25.180)
#Mean MSE: 236.620 (25.180)
#Mean Var. Explained: 0.084 (0.067)
#R-sq: 0.060 (0.072)
#MSE: 111362.78
#RMSE: 333.71
#R-sq: 0.14

# Take 2
#Mean MAE: 238.385 (23.135)
#Mean MSE: 238.385 (23.135)
#Mean Var. Explained: 0.083 (0.115)
#R-sq: 0.074 (0.114)
#MSE: 107278.62
#RMSE: 327.53
#R-sq: 0.17
#Total time: 18.04

XGB version: 1.7.6
Fitting 5 folds for each of 100 candidates, totalling 500 fits
{'reg_lambda': 100, 'reg_alpha': 10, 'objective': 'reg:squarederror', 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.1, 'gamma': 0.0}
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.0, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)
Mean MAE: 238.385 (23.135)
Mean MSE: 23

#### References

Mostly tutorials & blogs.

https://www.youtube.com/watch?v=OQKQHNCVf5k

https://www.youtube.com/watch?v=GrJP9FLV3FE&t=2167s

https://datascience.stackexchange.com/questions/16342/unbalanced-multiclass-data-with-xgboost

https://mljar.com/blog/xgboost-save-load-python/

https://machinelearningmastery.com/xgboost-for-regression/

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

https://www.datatechnotes.com/2019/06/regression-example-with-xgbregressor-in.html

https://github.com/parrt/dtreeviz

https://stackoverflow.com/questions/37627923/how-to-get-feature-importance-in-xgboost

https://towardsdatascience.com/be-careful-when-interpreting-your-features-importance-in-xgboost-6e16132588e7

https://scikit-learn.org/stable/modules/partial_dependence.html

https://scikit-learn.org/stable/auto_examples/inspection/plot_partial_dependence.html#way-partial-dependence-with-different-models

https://mljar.com/blog/xgboost-early-stopping/

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

https://github.com/parrt/dtreeviz



